Conda environment (for when Dan forgets): speech-emotion

Code from <a href="https://data-flair.training/blogs/python-mini-project-speech-emotion-recognition/">here</a>.

Goal is to deploy model in an Android app, so I'm making sure the code is actually reproducible here.

In [1]:
!ls

index.ipynb                             speech-emotion-recognition-ravdess-data


In [2]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [3]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [4]:
#DataFlair - Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [5]:
#DataFlair - Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

#load_data()

In [30]:
#DataFlair - Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

y_train = pd.get_dummies(pd.DataFrame(y_train, columns=['label']), prefix='', prefix_sep='')
y_test = pd.get_dummies(pd.DataFrame(y_test, columns=['label']), prefix='', prefix_sep='')

#DataFlair - Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))
#DataFlair - Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

(576, 192)
Features extracted: 180


In [7]:
#DataFlair - Initialize the Multi Layer Perceptron Classifier
import tqdm
tests = []
for test in tqdm.trange(1, 100):
    model=MLPClassifier(alpha=0.01,                # L2 penalty parameter (default 0.0001)          
                        batch_size=256,            # size of minibatch (default: min(200, n_samples))
                        epsilon=1e-08,             # value for numerical stability in adam
                        hidden_layer_sizes=(300,), # sizes (number of neurons) in the hidden layers
                        learning_rate='adaptive',  # fixed at learning_rate_init unless loss isn't decreasing, 
                        max_iter=500)              # maximum number of iterations to perform

    #DataFlair - Train the model
    model.fit(x_train,y_train)

    #DataFlair - Predict for the test set
    y_pred=model.predict(x_test)

    #DataFlair - Calculate the accuracy of our model
    accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

    #DataFlair - Print the accuracy
    #print("Accuracy: {:.2f}%".format(accuracy*100))
    tests.append(accuracy)

100%|██████████| 99/99 [01:17<00:00,  1.28it/s]


In [8]:
model.get_params()

{'activation': 'relu',
 'alpha': 0.01,
 'batch_size': 256,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (300,),
 'learning_rate': 'adaptive',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 500,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [9]:
# the ith element in the list represents the weight metrix corresponding to layer i
# 
for level, layer in enumerate(model.coefs_):
    print(len(layer), 'coefficients in layer', level)

180 coefficients in layer 0
300 coefficients in layer 1


In [10]:
for level, layer in enumerate(model.intercepts_):
    print(len(layer), 'intercepts in layer', level)

300 intercepts in layer 0
4 intercepts in layer 1


In [41]:
import tensorflow as tf

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=300, 
                                activation='relu', 
                                input_dim=len(x_train[0])))
model.add(tf.keras.layers.Dense(units=4, 
                                activation='softmax'))
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

history = model.fit(x_train, y_train,
                    epochs=500,
                    batch_size=256,
                    validation_split=0.1)

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 300)               54300     
_________________________________________________________________
dense_25 (Dense)             (None, 4)                 1204      
Total params: 55,504
Trainable params: 55,504
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
2/2 [==============================] - 0s 51ms/step - loss: 20.5386 - accuracy: 0.2717 - val_loss: 10.2816 - val_accuracy: 0.3448
Epoch 2/500
2/2 [==============================] - 0s 7ms/step - loss: 10.3018 - accuracy: 0.2174 - val_loss: 8.1802 - val_accuracy: 0.2500
Epoch 3/500
2/2 [==============================] - 0s 8ms/step - loss: 9.2867 - accuracy: 0.3217 - val_loss: 12.3096 - val_accuracy: 0.2672
Epoch 4/500
2/2 [==============================] - 0s 9ms/step - loss: 12.0746 - accuracy

2/2 [==============================] - 0s 9ms/step - loss: 0.6915 - accuracy: 0.7261 - val_loss: 0.9857 - val_accuracy: 0.6293
Epoch 56/500
2/2 [==============================] - 0s 8ms/step - loss: 0.6915 - accuracy: 0.7261 - val_loss: 0.9668 - val_accuracy: 0.6293
Epoch 57/500
2/2 [==============================] - 0s 8ms/step - loss: 0.6882 - accuracy: 0.7304 - val_loss: 0.9764 - val_accuracy: 0.6034
Epoch 58/500
2/2 [==============================] - 0s 8ms/step - loss: 0.6854 - accuracy: 0.7304 - val_loss: 1.0205 - val_accuracy: 0.6034
Epoch 59/500
2/2 [==============================] - 0s 9ms/step - loss: 0.6817 - accuracy: 0.7087 - val_loss: 1.0007 - val_accuracy: 0.5948
Epoch 60/500
2/2 [==============================] - 0s 8ms/step - loss: 0.6770 - accuracy: 0.7239 - val_loss: 0.9687 - val_accuracy: 0.6121
Epoch 61/500
2/2 [==============================] - 0s 8ms/step - loss: 0.6646 - accuracy: 0.7370 - val_loss: 1.0186 - val_accuracy: 0.5862
Epoch 62/500
2/2 [===============

2/2 [==============================] - 0s 9ms/step - loss: 0.5201 - accuracy: 0.8217 - val_loss: 1.1020 - val_accuracy: 0.6379
Epoch 114/500
2/2 [==============================] - 0s 8ms/step - loss: 0.5302 - accuracy: 0.7957 - val_loss: 1.0854 - val_accuracy: 0.6207
Epoch 115/500
2/2 [==============================] - 0s 8ms/step - loss: 0.5017 - accuracy: 0.8239 - val_loss: 1.0565 - val_accuracy: 0.6466
Epoch 116/500
2/2 [==============================] - 0s 8ms/step - loss: 0.5160 - accuracy: 0.8217 - val_loss: 1.1353 - val_accuracy: 0.6121
Epoch 117/500
2/2 [==============================] - 0s 8ms/step - loss: 0.5034 - accuracy: 0.8196 - val_loss: 1.1247 - val_accuracy: 0.6207
Epoch 118/500
2/2 [==============================] - 0s 7ms/step - loss: 0.5096 - accuracy: 0.8261 - val_loss: 1.0653 - val_accuracy: 0.6293
Epoch 119/500
2/2 [==============================] - 0s 8ms/step - loss: 0.5038 - accuracy: 0.8152 - val_loss: 1.1173 - val_accuracy: 0.6379
Epoch 120/500
2/2 [========

2/2 [==============================] - 0s 7ms/step - loss: 0.3975 - accuracy: 0.8739 - val_loss: 1.2488 - val_accuracy: 0.6552
Epoch 171/500
2/2 [==============================] - 0s 7ms/step - loss: 0.4021 - accuracy: 0.8739 - val_loss: 1.2570 - val_accuracy: 0.6379
Epoch 172/500
2/2 [==============================] - 0s 7ms/step - loss: 0.3914 - accuracy: 0.8761 - val_loss: 1.2893 - val_accuracy: 0.6379
Epoch 173/500
2/2 [==============================] - 0s 7ms/step - loss: 0.3854 - accuracy: 0.8804 - val_loss: 1.2689 - val_accuracy: 0.6552
Epoch 174/500
2/2 [==============================] - 0s 8ms/step - loss: 0.3816 - accuracy: 0.8826 - val_loss: 1.2620 - val_accuracy: 0.6552
Epoch 175/500
2/2 [==============================] - 0s 7ms/step - loss: 0.3825 - accuracy: 0.8696 - val_loss: 1.3037 - val_accuracy: 0.6552
Epoch 176/500
2/2 [==============================] - 0s 8ms/step - loss: 0.3810 - accuracy: 0.8870 - val_loss: 1.2561 - val_accuracy: 0.6810
Epoch 177/500
2/2 [========

Epoch 228/500
2/2 [==============================] - 0s 8ms/step - loss: 0.2723 - accuracy: 0.9304 - val_loss: 1.4580 - val_accuracy: 0.7155
Epoch 229/500
2/2 [==============================] - 0s 8ms/step - loss: 0.2759 - accuracy: 0.9261 - val_loss: 1.4446 - val_accuracy: 0.6897
Epoch 230/500
2/2 [==============================] - 0s 8ms/step - loss: 0.2649 - accuracy: 0.9413 - val_loss: 1.5017 - val_accuracy: 0.6897
Epoch 231/500
2/2 [==============================] - 0s 8ms/step - loss: 0.2797 - accuracy: 0.9304 - val_loss: 1.4746 - val_accuracy: 0.6897
Epoch 232/500
2/2 [==============================] - 0s 7ms/step - loss: 0.2782 - accuracy: 0.9152 - val_loss: 1.4527 - val_accuracy: 0.7069
Epoch 233/500
2/2 [==============================] - 0s 7ms/step - loss: 0.2716 - accuracy: 0.9304 - val_loss: 1.4764 - val_accuracy: 0.6983
Epoch 234/500
2/2 [==============================] - 0s 8ms/step - loss: 0.2701 - accuracy: 0.9217 - val_loss: 1.4592 - val_accuracy: 0.6810
Epoch 235/500

Epoch 286/500
2/2 [==============================] - 0s 8ms/step - loss: 0.1768 - accuracy: 0.9630 - val_loss: 1.6590 - val_accuracy: 0.7069
Epoch 287/500
2/2 [==============================] - 0s 9ms/step - loss: 0.1864 - accuracy: 0.9630 - val_loss: 1.6720 - val_accuracy: 0.6897
Epoch 288/500
2/2 [==============================] - 0s 8ms/step - loss: 0.1808 - accuracy: 0.9587 - val_loss: 1.6617 - val_accuracy: 0.6983
Epoch 289/500
2/2 [==============================] - 0s 8ms/step - loss: 0.1760 - accuracy: 0.9674 - val_loss: 1.6560 - val_accuracy: 0.7069
Epoch 290/500
2/2 [==============================] - 0s 8ms/step - loss: 0.1802 - accuracy: 0.9652 - val_loss: 1.6695 - val_accuracy: 0.6983
Epoch 291/500
2/2 [==============================] - 0s 8ms/step - loss: 0.1755 - accuracy: 0.9609 - val_loss: 1.6657 - val_accuracy: 0.6983
Epoch 292/500
2/2 [==============================] - 0s 8ms/step - loss: 0.1683 - accuracy: 0.9717 - val_loss: 1.6740 - val_accuracy: 0.6810
Epoch 293/500

Epoch 344/500
2/2 [==============================] - 0s 8ms/step - loss: 0.1198 - accuracy: 0.9870 - val_loss: 1.8172 - val_accuracy: 0.7069
Epoch 345/500
2/2 [==============================] - 0s 8ms/step - loss: 0.1192 - accuracy: 0.9826 - val_loss: 1.8228 - val_accuracy: 0.7069
Epoch 346/500
2/2 [==============================] - 0s 7ms/step - loss: 0.1185 - accuracy: 0.9848 - val_loss: 1.8234 - val_accuracy: 0.7241
Epoch 347/500
2/2 [==============================] - 0s 8ms/step - loss: 0.1164 - accuracy: 0.9870 - val_loss: 1.8271 - val_accuracy: 0.7241
Epoch 348/500
2/2 [==============================] - 0s 8ms/step - loss: 0.1177 - accuracy: 0.9891 - val_loss: 1.8298 - val_accuracy: 0.7155
Epoch 349/500
2/2 [==============================] - 0s 8ms/step - loss: 0.1150 - accuracy: 0.9870 - val_loss: 1.8278 - val_accuracy: 0.7069
Epoch 350/500
2/2 [==============================] - 0s 8ms/step - loss: 0.1165 - accuracy: 0.9870 - val_loss: 1.8295 - val_accuracy: 0.7155
Epoch 351/500

2/2 [==============================] - 0s 7ms/step - loss: 0.0835 - accuracy: 1.0000 - val_loss: 1.9697 - val_accuracy: 0.7069
Epoch 403/500
2/2 [==============================] - 0s 8ms/step - loss: 0.0835 - accuracy: 0.9957 - val_loss: 1.9554 - val_accuracy: 0.7069
Epoch 404/500
2/2 [==============================] - 0s 8ms/step - loss: 0.0824 - accuracy: 1.0000 - val_loss: 1.9546 - val_accuracy: 0.6983
Epoch 405/500
2/2 [==============================] - 0s 8ms/step - loss: 0.0816 - accuracy: 0.9978 - val_loss: 1.9860 - val_accuracy: 0.7069
Epoch 406/500
2/2 [==============================] - 0s 8ms/step - loss: 0.0831 - accuracy: 0.9957 - val_loss: 1.9912 - val_accuracy: 0.7155
Epoch 407/500
2/2 [==============================] - 0s 8ms/step - loss: 0.0801 - accuracy: 0.9935 - val_loss: 1.9575 - val_accuracy: 0.7155
Epoch 408/500
2/2 [==============================] - ETA: 0s - loss: 0.0818 - accuracy: 1.00 - 0s 8ms/step - loss: 0.0830 - accuracy: 0.9978 - val_loss: 1.9744 - val_ac

2/2 [==============================] - 0s 9ms/step - loss: 0.0595 - accuracy: 1.0000 - val_loss: 2.0900 - val_accuracy: 0.6897
Epoch 460/500
2/2 [==============================] - 0s 9ms/step - loss: 0.0592 - accuracy: 1.0000 - val_loss: 2.0763 - val_accuracy: 0.6983
Epoch 461/500
2/2 [==============================] - 0s 9ms/step - loss: 0.0581 - accuracy: 1.0000 - val_loss: 2.0822 - val_accuracy: 0.6983
Epoch 462/500
2/2 [==============================] - 0s 9ms/step - loss: 0.0593 - accuracy: 1.0000 - val_loss: 2.0902 - val_accuracy: 0.6983
Epoch 463/500
2/2 [==============================] - 0s 9ms/step - loss: 0.0586 - accuracy: 1.0000 - val_loss: 2.0861 - val_accuracy: 0.6983
Epoch 464/500
2/2 [==============================] - 0s 8ms/step - loss: 0.0580 - accuracy: 1.0000 - val_loss: 2.0974 - val_accuracy: 0.6897
Epoch 465/500
2/2 [==============================] - 0s 8ms/step - loss: 0.0595 - accuracy: 1.0000 - val_loss: 2.1058 - val_accuracy: 0.6983
Epoch 466/500
2/2 [========